# Road Sign detection model

In [ ]:
# useful libraries
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
print('Tensorflow version :', tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

%matplotlib inline

### Load the data and check shapes

In [ ]:
# load data
with open("./traffic-signs-data/train.p", mode='rb') as training_data:
    train = pickle.load(training_data)
with open("./traffic-signs-data/valid.p", mode='rb') as validation_data:
    valid = pickle.load(validation_data)
with open("./traffic-signs-data/test.p", mode='rb') as testing_data:
    test = pickle.load(testing_data)

In [ ]:
# label and features
x_train, y_train = train['features'], train['labels']
x_validation, y_validation = valid['features'], valid['labels']
x_test, y_test = test['features'], test['labels']

# check the shape of the data
x_shape = x_train.shape
x_val_shape = x_validation.shape

print("x_training shape :", x_shape)
print("x_validation shape :", x_val_shape)

In [ ]:
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train)

### Convert to grayscale and normalization of the data


In [ ]:
# convert color images to grayscale using averaging method (np.sum)
x_train_gray = np.sum(x_train, axis=3, keepdims=True)/3
x_test_gray  = np.sum(x_test, axis=3, keepdims=True)/3
x_validation_gray  = np.sum(x_validation, axis=3, keepdims=True)/3

# normalization of the data
x_train_gray_norm = (x_train_gray)/255.0
x_test_gray_norm = (x_test_gray)/255.0
x_validation_gray_norm = (x_validation_gray)/255.0

# one-hot encoding classes
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_validation = to_categorical(y_validation)

In [ ]:
# visualize the data
i = 610
plt.subplot(1,2,1)
plt.imshow(x_train_gray_norm[i].squeeze(), cmap='gray')
plt.subplot(1,2,2)
plt.imshow(x_train[i])

### Model architecture

After normalization of the data, we've one-hot encoded the 43 different classes of road signs. Now we'll define a model architecture to train our network and to prevent overfitting.

In [ ]:
from tensorflow.keras import regularizers

# Model architecture
def cnn_model_architecture(kernel_size, input_shape, batch_size, epochs):
            
    cnn_model = tf.keras.Sequential()
           
    cnn_model.add(layers.Conv2D(32, kernel_size, activation = 'relu', kernel_regularizer=regularizers.l2(0.001), input_shape=input_shape)) 
    cnn_model.add(layers.MaxPooling2D(pool_size=(2,2))) 
    cnn_model.add(layers.Dropout(0.5))
    cnn_model.add(layers.Conv2D(64, kernel_size, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=input_shape))      
    cnn_model.add(layers.MaxPooling2D(pool_size=(2,2))) 
    cnn_model.add(layers.Conv2D(64, kernel_size, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=input_shape)) 
    cnn_model.add(layers.MaxPooling2D(pool_size=(2,2)))        
    cnn_model.add(layers.Dropout(0.5))

    cnn_model.add(layers.Flatten())   
    cnn_model.add(layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))) 
    cnn_model.add(layers.Dropout(0.5))   
    cnn_model.add(layers.Dense(43, activation='softmax'))
    
    return cnn_model   

In [ ]:
# Compiling the model 
def compile_model():
    cnn_model_architecture(kernel_size, input_shape, batch_size, epochs)    
    cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    return cnn_model

### Compile the model

In [ ]:
kernel_size = (3,3)
input_shape = x_train_gray_norm[1].shape
batch_size = 100
epochs = 300

cnn_model = cnn_model_architecture(kernel_size, input_shape, batch_size, epochs)
cnn_model = compile_model()

### Train and evaluate the model

In [ ]:
def train_model(model, x_train, y_train, x_validation, y_validation):   

    # train the model    
    history = model.fit(x_train,
                        y_train,
                        batch_size= batch_size,
                        epochs = epochs, 
                        steps_per_epoch = len(x_train) / batch_size,
                        verbose=1, 
                        validation_data= (x_validation, y_validation))
    return history

In [ ]:
# train and evaluate the model
history = train_model(cnn_model,
                                   x_train_gray_norm, 
                                   y_train, 
                                   x_validation_gray_norm, 
                                   y_validation)

In [ ]:
# evaluate the model
score = cnn_model.evaluate(x_test_gray_norm, y_test, verbose=1)
print('\nTest loss : {:.4f}'.format(score[0]))
print('Test Accuracy : {:.4f}'.format(score[1]))

In [ ]:
# plot loss _values
history_dict = history.history

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values)+ 1)
line1 = plt.plot(epochs, val_loss_values, label = 'Validation Loss')
line2 = plt.plot(epochs, loss_values, label= 'Training Loss')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker= '4', markersize=10.0)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# plot accuracy 

acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']
epochs = range(1, len(loss_values)+ 1)
line1 = plt.plot(epochs, val_acc_values, label = 'Validation Accuracy')
line2 = plt.plot(epochs, acc_values, label= 'Training Accuracy')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker= '4', markersize=10.0)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# save model
cnn_model.save("./road_sign_model_2.h5")

In [32]:
from tensorflow.keras import backend as K
K.clear_session()

In [ ]:
%%javascript
<!-- Save the notebook -->
IPython.notebook.save_checkpoint();

In [ ]:
%%javascript
IPython.notebook.session.delete();
window.onbeforeunload = null
setTimeout(function() { window.close(); }, 1000);